In [80]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('words')


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tuning_model_class import w2v_tuning
from transformers import  ElectraTokenizerFast
from nltk.corpus import wordnet, stopwords, words
import json
import pandas as pd

In [62]:
vocabulary = words.words()

In [63]:
w2vmodel = w2v_tuning()
w2vmodel.set_vector_path("w2v.42B.300d.txt",'GloVe.wordvectors')
w2vmodel.CreateW2V_Model()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
load pre-train model


In [64]:
w2vmodel.model.wv.most_similar('event',topn=5)

[('events', 0.8563973903656006),
 ('venue', 0.6610988974571228),
 ('weekend', 0.6492050290107727),
 ('celebration', 0.6340488195419312),
 ('show', 0.620791494846344)]

In [65]:
# db_schema = ['zipcode','state','company','product','date']
db_schema = ['id','name','age','song','show','order']

In [66]:
target_word = 'How many singer in ABC concert?'
# target_word = 'Who play LOVE in ABC tour?'
target_word = 'Which name of singer that play in event?'

In [67]:
def preprocess_question(text:str):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [WordNetLemmatizer().lemmatize(word, wordnet.NOUN) for word in tokens if (word not in stop_words and word.isalnum())]
    return filtered_tokens

In [68]:
def tokenize_column(col_list:list):
    tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")
    token_col = [tokenizer.tokenize(col_name) for col_name in col_list]
    return token_col

In [69]:
for word in preprocess_question(target_word):
    similarities = {}
    for w in db_schema:
        similarities[w] = w2vmodel.model.wv.similarity(word,w)

    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    print(f"'{word}' similar '{sorted_similarities[0][0]} {sorted_similarities[0][1]:.2f}'")

'name' similar 'name 1.00'
'singer' similar 'song 0.63'
'play' similar 'song 0.52'
'event' similar 'show 0.62'


In [70]:
# can can redirect to fullcolumn name (with key)

In [71]:
db_id = "concert_singer"
questions = []

with open("spider/dev.json", 'r') as json_file:
    data = json.load(json_file)

for element in data:
    if element['db_id'] == db_id:
        questions.append(element['question'])

In [72]:
import sqlite3

# Specify the full path to the SQLite database file
db_file_path = 'data/concert_singer/concert_singer.sqlite'
db_dict = {}
col_to_tables = {}

# Connect to the database
connection = sqlite3.connect(db_file_path)

# Create a cursor object
cursor = connection.cursor()

# Query the SQLite master table to get the list of table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
table_names = cursor.fetchall()

# Iterate through the table names
for table in table_names:
    table_name = table[0]
    print(f"Table: {table_name}")

    # Query the PRAGMA statement to get column information for the current table
    cursor.execute(f"PRAGMA table_info({table_name})")

    # Fetch column information
    column_info = cursor.fetchall()

    # Extract and print column names for the current table
    column_names = [col[1] for col in column_info]
    print("Columns:", column_names)
    # print("Tokenize columns:", tokenize_column(column_names))
    db_dict[table_name] = column_names
    
    for col in column_names:
        col_to_tables[col] = table_name

    print("\n")

# Close the cursor and the connection
cursor.close()
connection.close()


Table: stadium
Columns: ['Stadium_ID', 'Location', 'Name', 'Capacity', 'Highest', 'Lowest', 'Average']


Table: singer
Columns: ['Singer_ID', 'Name', 'Country', 'Song_Name', 'Song_release_year', 'Age', 'Is_male']


Table: concert
Columns: ['concert_ID', 'concert_Name', 'Theme', 'Stadium_ID', 'Year']


Table: singer_in_concert
Columns: ['concert_ID', 'Singer_ID']




In [73]:
db_dict

{'stadium': ['Stadium_ID',
  'Location',
  'Name',
  'Capacity',
  'Highest',
  'Lowest',
  'Average'],
 'singer': ['Singer_ID',
  'Name',
  'Country',
  'Song_Name',
  'Song_release_year',
  'Age',
  'Is_male'],
 'concert': ['concert_ID', 'concert_Name', 'Theme', 'Stadium_ID', 'Year'],
 'singer_in_concert': ['concert_ID', 'Singer_ID']}

In [74]:
col_to_tables

{'Stadium_ID': 'concert',
 'Location': 'stadium',
 'Name': 'singer',
 'Capacity': 'stadium',
 'Highest': 'stadium',
 'Lowest': 'stadium',
 'Average': 'stadium',
 'Singer_ID': 'singer_in_concert',
 'Country': 'singer',
 'Song_Name': 'singer',
 'Song_release_year': 'singer',
 'Age': 'singer',
 'Is_male': 'singer',
 'concert_ID': 'singer_in_concert',
 'concert_Name': 'concert',
 'Theme': 'concert',
 'Year': 'concert'}

In [75]:
questions[:10]

['How many singers do we have?',
 'What is the total number of singers?',
 'Show name, country, age for all singers ordered by age from the oldest to the youngest.',
 'What are the names, countries, and ages for every singer in descending order of age?',
 'What is the average, minimum, and maximum age of all singers from France?',
 'What is the average, minimum, and maximum age for all French singers?',
 'Show the name and the release year of the song by the youngest singer.',
 'What are the names and release years for all the songs of the youngest singer?',
 'What are all distinct countries where singers above age 20 are from?',
 'What are  the different countries with singers above age 20?']

In [76]:
for q in questions[:10]:
    q_tokens = preprocess_question(q)
    print(q_tokens)

['many', 'singer']
['total', 'number', 'singer']
['show', 'name', 'country', 'age', 'singer', 'ordered', 'age', 'oldest', 'youngest']
['name', 'country', 'age', 'every', 'singer', 'descending', 'order', 'age']
['average', 'minimum', 'maximum', 'age', 'singer', 'france']
['average', 'minimum', 'maximum', 'age', 'french', 'singer']
['show', 'name', 'release', 'year', 'song', 'youngest', 'singer']
['name', 'release', 'year', 'song', 'youngest', 'singer']
['distinct', 'country', 'singer', 'age', '20']
['different', 'country', 'singer', 'age', '20']


In [ ]:
def compare_word_col(df, new_qword=False):
    
    return

In [ ]:
def map_column_table(db_dict,col_to_table,column_name):
    return

In [79]:
q = questions[0]
print("Question:", q)

q_tokens = preprocess_question(q)
print("Question tokens:", q_tokens)



Question: How many singers do we have?
Question tokens: ['many', 'singer']
